# Email Agent Pipeline with LangGraph

This notebook demonstrates building a simple email agent using LangGraph.

## Overview
- **LangGraph**: Agent orchestration and workflow
- **OpenAI**: LLM for natural language understanding

## Setup and Imports

In [10]:
import os
from typing import Annotated, Literal, TypedDict, List, Dict, Any
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv

load_dotenv()

True

## Define Agent State

Create the state structure for our email agent using TypedDict.

In [11]:
class EmailAgentState(TypedDict):
    messages: Annotated[list, add_messages]

## Create Email Processing Tools

Define tools for email operations: compose, categorize, extract action items, and draft replies.

In [15]:
@tool
def compose_email(recipient: str, subject: str, tone: str = "professional") -> str:
    """
    Compose a professional email with the given recipient, subject, and tone.
    
    Args:
        recipient: The email recipient name
        subject: The subject of the email
        tone: The tone of the email (professional, friendly, formal, casual)
    """
    return f"""Email composed successfully!

To: {recipient}
Subject: {subject}
Tone: {tone}

[Ready for AI to generate the full email content based on context]
"""


@tool
def categorize_email(email_content: str) -> str:
    """
    Categorize an email as urgent, spam, personal, work, or promotional.
    
    Args:
        email_content: The content of the email to categorize
    """
    email_lower = email_content.lower()
    
    if any(word in email_lower for word in ["urgent", "asap", "immediately", "deadline"]):
        category = "urgent"
    elif any(word in email_lower for word in ["lottery", "winner", "click here", "free money", "congratulations you won"]):
        category = "spam"
    elif any(word in email_lower for word in ["meeting", "project", "report", "quarterly", "client"]):
        category = "work"
    elif any(word in email_lower for word in ["offer", "discount", "sale", "buy now", "limited time"]):
        category = "promotional"
    else:
        category = "personal"
    
    return f"Email categorized as: {category.upper()}"


@tool
def extract_action_items(email_content: str) -> str:
    """
    Extract action items and tasks from an email.
    
    Args:
        email_content: The content of the email
    """
    return f"""Action items will be extracted from the email:

Email content analyzed: {len(email_content)} characters

[AI will identify specific action items, deadlines, and tasks]
"""


@tool
def draft_reply(original_email: str, reply_type: str = "acknowledge") -> str:
    """
    Draft a reply to an email.
    
    Args:
        original_email: The original email content
        reply_type: Type of reply (acknowledge, decline, accept, request_info)
    """
    return f"""Reply drafted (Type: {reply_type}):

Responding to: {original_email[:100]}...

[AI will generate an appropriate reply based on the original email and reply type]
"""


# Collect all tools
tools = [compose_email, categorize_email, extract_action_items, draft_reply]

print(f"✅ Created {len(tools)} email processing tools:")
for t in tools:
    print(f"   - {t.name}: {t.description.split('.')[0]}")

✅ Created 4 email processing tools:
   - compose_email: Compose a professional email with the given recipient, subject, and tone
   - categorize_email: Categorize an email as urgent, spam, personal, work, or promotional
   - extract_action_items: Extract action items and tasks from an email
   - draft_reply: Draft a reply to an email


## Build the LangGraph Workflow

Create the agent workflow with nodes and edges for email processing.

In [16]:
model = ChatOpenAI(model="gpt-4.1", temperature=0.7)
model_with_tools = model.bind_tools(tools)

def should_continue(state: EmailAgentState) -> Literal["tools", "__end__"]:
    """Determine if we should continue to tools or end."""
    messages = state["messages"]
    last_message = messages[-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return END


def call_model(state: EmailAgentState):
    """Call the LLM with the current state."""
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(EmailAgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode(tools))

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

email_agent = workflow.compile()

print("✅ LangGraph workflow built successfully!")
print("\nWorkflow structure:")
print("  START → agent → [conditional] → tools → agent → END")

✅ LangGraph workflow built successfully!

Workflow structure:
  START → agent → [conditional] → tools → agent → END


## Test the Agent

Let's test the email agent with different scenarios.

In [17]:
def run_email_agent(user_input: str):
    """Run the email agent with user input."""
    print(f"\n{'='*70}")
    print(f"🗣️  USER: {user_input}")
    print(f"{'='*70}\n")
    
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    for output in email_agent.stream(inputs):
        for key, value in output.items():
            if key == "agent":
                messages = value.get("messages", [])
                if messages:
                    last_message = messages[-1]
                    if isinstance(last_message, AIMessage) and last_message.content:
                        print(f"🤖 AGENT: {last_message.content}\n")
            elif key == "tools":
                print(f"🔧 Tools executed\n")
    
    return inputs

test1 = run_email_agent(
    "Compose a professional email to Sarah Johnson about the Q4 budget review meeting scheduled for next Tuesday."
)


🗣️  USER: Compose a professional email to Sarah Johnson about the Q4 budget review meeting scheduled for next Tuesday.

🔧 Tools executed

🤖 AGENT: Here is a professional email you can send to Sarah Johnson regarding the Q4 budget review meeting:

---

Subject: Q4 Budget Review Meeting Scheduled for Next Tuesday

Dear Sarah,

I hope this message finds you well. I am writing to confirm that the Q4 budget review meeting is scheduled for next Tuesday. During this meeting, we will discuss our financial performance for the quarter, review budget allocations, and address any outstanding questions or concerns.

Please let me know if you have any specific topics you would like to add to the agenda or if there are any scheduling conflicts. 

Looking forward to your participation.

Best regards,  
[Your Name]  

---

Let me know if you want to add any specific details or attachments!



In [18]:
test2 = run_email_agent(
    "Categorize this email: 'URGENT: Client presentation deadline moved to tomorrow morning. Please review the slides ASAP.'"
)


🗣️  USER: Categorize this email: 'URGENT: Client presentation deadline moved to tomorrow morning. Please review the slides ASAP.'

🔧 Tools executed

🤖 AGENT: This email is categorized as URGENT. It contains a pressing deadline and requires immediate attention.



In [19]:
test3 = run_email_agent(
    "Extract action items from: 'Hi team, for the product launch: 1) Finalize the marketing materials by Friday, 2) Schedule user testing sessions, 3) Prepare demo environment, 4) Send invitation emails to stakeholders.'"
)


🗣️  USER: Extract action items from: 'Hi team, for the product launch: 1) Finalize the marketing materials by Friday, 2) Schedule user testing sessions, 3) Prepare demo environment, 4) Send invitation emails to stakeholders.'

🔧 Tools executed

🤖 AGENT: Here are the extracted action items from the email:

1. Finalize the marketing materials by Friday.
2. Schedule user testing sessions.
3. Prepare the demo environment.
4. Send invitation emails to stakeholders.

Let me know if you need these organized further or assigned to specific team members!



In [20]:
test4 = run_email_agent(
    "Draft a polite decline reply to: 'Would you like to join our weekly team standup on Mondays at 8 AM?'"
)


🗣️  USER: Draft a polite decline reply to: 'Would you like to join our weekly team standup on Mondays at 8 AM?'

🔧 Tools executed

🤖 AGENT: Here is a polite decline reply you can use:

Thank you very much for the invitation to join your weekly team standup on Mondays at 8 AM. I appreciate being considered, but unfortunately, I won’t be able to attend at that time. Please do keep me in the loop for any updates or information I might need. Thank you again for understanding.



## Export Agent Function for API

Create a reusable function that can be used by the FastAPI backend.

In [25]:
def process_email_request(user_input: str) -> Dict[str, Any]:
    """
    Process an email request and return the result.
    This function can be used by the FastAPI backend.
    
    Args:
        user_input: The user's request/query
        
    Returns:
        Dictionary with response and metadata
    """
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    response_text = ""
    tool_calls = []
    
    for output in email_agent.stream(inputs):
        for key, value in output.items():
            if key == "agent":
                messages = value.get("messages", [])
                if messages:
                    last_message = messages[-1]
                    if isinstance(last_message, AIMessage):
                        if last_message.content:
                            response_text = last_message.content
                        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                            tool_calls = [tc.get('name', 'unknown') for tc in last_message.tool_calls]
    
    return {
        "response": response_text,
        "tools_used": tool_calls,
        "status": "success"
    }


result = process_email_request("Compose a quick thank you email to the team")
print("✅ Agent function ready for API integration!")
print(f"\nSample result structure:")
print(f"  - response: {len(result['response'])} characters")
print(f"  - tools_used: {result['tools_used']}")
print(f"  - status: {result['status']}")

✅ Agent function ready for API integration!

Sample result structure:
  - response: 340 characters
  - tools_used: ['compose_email']
  - status: success


In [26]:
print(result['response'])

Here’s a quick thank you email you can send to your team:

---

Subject: Thank You!

Hi Team,

I just wanted to take a moment to thank everyone for your hard work and dedication. I truly appreciate all of your efforts and teamwork. Keep up the great work!

Best,  
[Your Name]

---

Let me know if you’d like to personalize or add anything!
